In [4]:
import pandas as pd
import numpy as np
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.utils import get_tmpfile
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC


df = pd.read_csv("../crimeBB/crimebb_preprocessed.csv")
df.dropna(subset=['Tags0'], inplace=True) # drop rows without labels
df.CVE = df.CVE.str.lower()
df = df[~df.Tags0.isin(['russian', '\\', 'others', 'doubt', 'error', 'market'])]
df.reset_index(inplace=True)
df.head()

,index,Unnamed: 0,Unnamed: 0.1,CVE,IdPost,Time,Likes,Content,obs,Tags0,Tags1,Tags2
0,0,0,1550.0,cve-2012-0500,7345.0,2012-03-06 00:28:00-03:00,NaN,Binary Analysis of Oracle Java CVE: 2012-0500 ...,NaN,weaponization,NaN,NaN
1,2,2,1263.0,cve-2013-0431,32622237.0,2013-05-13 22:24:00-03:00,NaN,Ok I've made threads about this before and non...,NaN,exploitation,NaN,NaN
2,4,4,1223.0,cve-2017-0199,56090050.0,2017-10-18 08:34:00-02:00,NaN,***CITING***[https://hackforums.net/showthread...,Trillium,exploitation,NaN,NaN
3,5,5,1617.0,cve-2015-0313,45426930.0,2015-03-27 00:01:00-03:00,NaN,Has CVE means not 0day\nIf you're talking abou...,NaN,weaponization,NaN,NaN
4,6,6,526.0,cve-2016-5195,53854632.0,2017-01-19 02:07:00-02:00,NaN,How To Get Root Access to Any Server\n\nHello ...,tutorial,PoC,exploitation,NaN


In [3]:
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import time
import multiprocessing
import gc

def filter_stopwords(words):
    stopWords = set(stopwords.words('english'))
    wordsFiltered = []
    for w in words:
        if w not in stopWords:
            wordsFiltered.append(w)
    return wordsFiltered

def removePunctuation(text):
    return text.translate(str.maketrans(punctuation, " "*len(punctuation)))

def stem(words):
    return [ps.stem(word) for word in words]

def writeLine(line):
    f.write(line + "\n")
    
def preprocessLine(line):
    line = removePunctuation(line)
    line = list(filter(lambda x: x not in stopWords, word_tokenize(line)))
    line = stem(line)
    line = " ".join(line)
    writeLine(line)

stopWords = set(stopwords.words('english'))
ps = nltk.stem.PorterStemmer()

files = ['hackforums_corpus.csv']
punctuation = string.punctuation
i = 1
with open("hackforums_corpus.txt", "w") as f:
    for file in files:
        chunks = pd.read_csv(file, names=['Content'], chunksize=1000000)
        for chunk in chunks:
            start = time.time()
            print("chunk: ", i)
            i += 1
            chunk['Content'] = chunk['Content'].astype(str)
            chunk.Content.str.lower().map(preprocessLine)
            del chunk
            gc.collect()
            print("took: ", time.time()-start)

chunk:  1
took:  751.7687385082245
chunk:  2
took:  745.759961605072
chunk:  3
took:  744.8197755813599
chunk:  4
took:  761.7826633453369
chunk:  5
took:  758.2635855674744
chunk:  6
took:  772.2487435340881
chunk:  7
took:  644.3198282718658


In [8]:
f = open("hackforums_corpus.txt", "a")
df.Content = df.Content.astype(str)
df.Content.str.lower().map(preprocessLine)

0       None
1       None
2       None
3       None
4       None
5       None
6       None
7       None
8       None
9       None
10      None
11      None
12      None
13      None
14      None
15      None
16      None
17      None
18      None
19      None
20      None
21      None
22      None
23      None
24      None
25      None
26      None
27      None
28      None
29      None
        ... 
2146    None
2147    None
2148    None
2149    None
2150    None
2151    None
2152    None
2153    None
2154    None
2155    None
2156    None
2157    None
2158    None
2159    None
2160    None
2161    None
2162    None
2163    None
2164    None
2165    None
2166    None
2167    None
2168    None
2169    None
2170    None
2171    None
2172    None
2173    None
2174    None
2175    None
Name: Content, Length: 2176, dtype: object

In [ ]:
f.close()

In [2]:
fname = "hackforums_corpus"
model = Doc2Vec(corpus_file=fname, vector_size=50, min_count=2, workers=4)

In [6]:
import smart_open
import gensim
fname = "hackforums_corpus"

def read_corpus(fname, tokens_only=False):
    with smart_open.open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            tokens = gensim.utils.simple_preprocess(line)
            if tokens_only:
                yield tokens
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

# corpus = list(read_corpus(fname))
model = Doc2Vec(read_corpus(fname), vector_size=300, min_count=2)
model.build_vocab(corpus)
model.train(corpus, total_examples=model.corpus_count, epochs=model.epochs)

leu corpus?


TypeError: You can't pass a generator as the documents argument. Try a sequence.

In [2]:
lbl = LabelEncoder()
df.Tags0 = lbl.fit_transform(df.Tags0.astype(str))
lbl.classes_

array(['PoC', 'exploitation', 'weaponization'], dtype=object)

In [4]:
from nltk import RegexpTokenizer
#tokenize

tk = RegexpTokenizer(r"[a-zA-Z]+")
df['ContentTokenized'] = pd.Series(['']*len(df))
df['Content'] = df['Content'].astype(str).str.lower()
for i in range(len(df)):
    words = tk.tokenize(df.at[i, 'Content'])
#     words = [word for word in nltk.word_tokenize(df.at[i, 'Content']) if word.isalnum()]
    df.at[i, 'ContentTokenized'] = filter_stopwords(words)

# Training Doc2Vec in whole dataset and evaluating its perfomance against the subset of posts citing CVEs